In [1]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

from implementations import *
from evaluation import *
import datetime

## Load the training data into feature matrix, class labels, and event ids:

In [2]:
from proj1_helpers import *
DATA_TRAIN_PATH = 'data/train.csv'
y, tX, ids = load_csv_data(DATA_TRAIN_PATH, sub_sample=False)

## Properties of dataset

In [3]:
from collections import Counter
Counter(y)

Counter({1.0: 85667, -1.0: 164333})

## Collect data

In [4]:
evaluation_data = []

## 1. Linear regression using gradient descent

In [5]:
#max_iters = [1]
#gammas = [0.1]
#poly_degree = 1

In [6]:
# Parameters
max_iters = [10, 100, 500, 750, 1000, 1500]
gammas = [0.001, 0.01, 0.049, 0.05, 0.053, 0.07, 0.1]
poly_degree = 10

In [ ]:
# Parameters
initial_w = np.array(np.zeros(30*poly_degree+1))

for max_iter in max_iters:
    for gamma in gammas:
        evaluation_result = cross_val(tX, y, 10, poly_degree, least_squares_GD, initial_w=initial_w, max_iters=max_iter, gamma=gamma)
                                    
        evaluation_data.append(evaluation_result)
        print(evaluation_result)

## 2. Linear regression using stochastic gradient descent

In [ ]:
# Parameters
initial_w = np.array(np.zeros(30*poly_degree+1))

for max_iter in max_iters:
    for gamma in gammas:
        evaluation_result = cross_val(tX, y, 10, poly_degree, least_squares_SGD, initial_w=initial_w, max_iters=max_iter, gamma=gamma)
        
        evaluation_data.append(evaluation_result)
        print(evaluation_result)

## 3. Least squares regression using normal equations

In [ ]:
evaluation_result = cross_val(tX, y, 10, poly_degree, least_squares)
        
evaluation_data.append(evaluation_result)
print(evaluation_result)

## 4. Ridge regression using normal equations

In [ ]:
# Parameters
lambda_ = [1, 10, 100, 500, 750, 1000, 1500]

for lambda_ in lambdas:
    evaluation_result = cross_val(tX, y, 10, poly_degree, ridge_regression, lambda_=lambda_)
        
    evaluation_data.append(evaluation_result)
    print(evaluation_result)

## 5. Logistic regression using gradient descent or SGD

In [ ]:
# Parameters
initial_w = np.array(np.zeros(30*poly_degree+1))

for max_iter in max_iters:
    for gamma in gammas:
        evaluation_result = cross_val(tX, y, 10, poly_degree, logistic_regression, max_iters=max_iter, gamma=gamma, initial_w=initial_w)
        
        evaluation_data.append(evaluation_result)
        print(evaluation_result)

## 6. Regularized logistic regression using gradient descent or SGD

In [ ]:
# Parameters
poly_degree = 10
initial_w = np.array(np.zeros(30*poly_degree+1))

for max_iter in max_iters:
    for gamma in gammas:
        for lambda_ in lambdas:
            evaluation_result = cross_val(tX, y, 10, poly_degree, reg_logistic_regression, max_iters=max_iter, gamma=gamma, initial_w=initial_w, lambda_=lambda_)

            evaluation_data.append(evaluation_result)
            print(evaluation_result)

## Final overview

In [ ]:
for el in evaluation_data:
    print(el)